In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

from sklearn import neighbors
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import json
with open("df_1hour_Feb.json", 'r', encoding='UTF-8') as f:
    F_data = json.load(f)

In [3]:
with open("df_1hour_Mar.json", 'r', encoding='UTF-8') as f:
    M_data = json.load(f)

In [4]:
COLUMNS = ['name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1', 'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2', 'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3', 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'kospi', 'kosdaq', 'trash', 'yesterday_closing_price', 'is_maximum', 'is_minimum', 'price_volatility', 'price_trend', 'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2', 'sell_minus_buy_3', 'is_price_gap_stable', 'price_gap_volatility', 'is_like_higher', 'volume_trend', 'post_num_trend', 'unique_id_trend', 'click_trend', 'price_increase', 'did_price_increase', 'did_price_033', 'did_price_100', 'did_price_150', 'kospi_ind', 'kosdaq_ind', 'time_slot', 'ko_inter', 'early_mor', 'morning', 'lunch', 'afternoon', 'late', 'mkt_change', 'alpha', 'per_now', 'kospi_1', 'kospi_2', 'kospi_3', 'kospi_answer', 'kosdaq_1', 'kosdaq_2', 'kosdaq_3', 'kosdaq_answer', 'kospi_trend', 'kosdaq_trend', 'kospi_increase', 'kosdaq_increase', 'market_increase', 'did_opening_price_increase', 'price_1_sq', 'price_dif_1_sq', 'sell_1_sq', 'buy_1_sq', 'volume_1_sq', 'variation_1_sq', 'post_num_1_sq', 'unique_id_1_sq', 'click_1_sq', 'like_1_sq', 'dislike_1_sq', 'price_2_sq', 'price_dif_2_sq', 'sell_2_sq', 'buy_2_sq', 'volume_2_sq', 'variation_2_sq', 'post_num_2_sq', 'unique_id_2_sq', 'click_2_sq', 'like_2_sq', 'dislike_2_sq', 'price_3_sq', 'price_dif_3_sq', 'sell_3_sq', 'buy_3_sq', 'volume_3_sq', 'variation_3_sq', 'post_num_3_sq', 'unique_id_3_sq', 'click_3_sq', 'like_3_sq', 'dislike_3_sq', 'mkt_cap_sq', 'yesterday_closing_price_sq', 'price_volatility_sq', 'price_trend_sq', 'average_price_volatility_sq', 'sell_minus_buy_1_sq', 'sell_minus_buy_2_sq', 'sell_minus_buy_3_sq', 'price_gap_volatility_sq', 'volume_trend_sq', 'post_num_trend_sq', 'unique_id_trend_sq', 'click_trend_sq', 'kospi_ind_sq', 'kosdaq_ind_sq', 'time_slot_sq', 'ko_inter_sq', 'mkt_change_sq', 'alpha_sq', 'per_now_sq', 'kospi_1_sq', 'kospi_2_sq', 'kospi_3_sq', 'kosdaq_1_sq', 'kosdaq_2_sq', 'kosdaq_3_sq', 'kospi_trend_sq', 'kosdaq_trend_sq']

In [5]:
df_F = pd.DataFrame(F_data, columns = COLUMNS)
df_M = pd.DataFrame(M_data, columns = COLUMNS)
df = pd.concat([df_F, df_M])

In [6]:
df = df.dropna(axis=0, how='any')

In [7]:
time_filter = (df['time'].str.startswith("2018-02-21")) | \
              (df['time'].str.startswith("2018-02-20")) | \
              (df['time'].str.startswith("2018-02-14")) 
        
train_df = df[time_filter].reset_index(drop = True)
test_df = df[~time_filter].reset_index(drop = True)

In [8]:
X_COL = ['price_1','price_dif_1','sell_1','buy_1','volume_1','variation_1','post_num_1','unique_id_1','click_1',
     'like_1','dislike_1','price_2','price_dif_2','sell_2','buy_2','volume_2','variation_2','post_num_2','unique_id_2',
     'click_2','like_2','dislike_2','price_3','price_dif_3','sell_3','buy_3','volume_3','variation_3','post_num_3',
     'unique_id_3','click_3','like_3','dislike_3','mkt_cap','kospi','kosdaq','trash','yesterday_closing_price',
     'is_maximum','is_minimum','price_volatility','price_trend','average_price_volatility','sell_minus_buy_1',
     'sell_minus_buy_2','sell_minus_buy_3','is_price_gap_stable','price_gap_volatility','is_like_higher',
     'volume_trend','post_num_trend','unique_id_trend','click_trend','kospi_ind','kosdaq_ind','time_slot',
     'ko_inter','early_mor','morning','lunch','afternoon','late','mkt_change','alpha','per_now','kospi_1','kospi_2',
     'kospi_3','kosdaq_1','kosdaq_2','kosdaq_3','kospi_trend','kosdaq_trend','did_opening_price_increase',
     'price_1_sq','price_dif_1_sq','sell_1_sq','buy_1_sq','volume_1_sq','variation_1_sq','post_num_1_sq',
     'unique_id_1_sq','click_1_sq','like_1_sq','dislike_1_sq','price_2_sq','price_dif_2_sq','sell_2_sq',
     'buy_2_sq','volume_2_sq','variation_2_sq','post_num_2_sq','unique_id_2_sq','click_2_sq','like_2_sq',
     'dislike_2_sq','price_3_sq','price_dif_3_sq','sell_3_sq','buy_3_sq','volume_3_sq','variation_3_sq',
     'post_num_3_sq','unique_id_3_sq','click_3_sq','like_3_sq','dislike_3_sq','mkt_cap_sq',
     'yesterday_closing_price_sq','price_volatility_sq','price_trend_sq','average_price_volatility_sq',
     'sell_minus_buy_1_sq','sell_minus_buy_2_sq','sell_minus_buy_3_sq','price_gap_volatility_sq',
     'volume_trend_sq','post_num_trend_sq','unique_id_trend_sq','click_trend_sq','kospi_ind_sq','kosdaq_ind_sq',
     'time_slot_sq','ko_inter_sq','mkt_change_sq','alpha_sq','per_now_sq','kospi_1_sq','kospi_2_sq','kospi_3_sq',
     'kosdaq_1_sq','kosdaq_2_sq','kosdaq_3_sq','kospi_trend_sq','kosdaq_trend_sq']

In [9]:
X = df[X_COL]
y = df['did_price_033']
X_train = train_df[X_COL]
X_test = test_df[X_COL]
y_train = train_df['did_price_033']
y_test = test_df['did_price_033']
y_test_in = test_df['price_increase']

In [10]:
X_train.shape

(5425, 135)

In [11]:
X_test.shape

(8171, 135)

In [12]:
def get_neighbour_mean(num):
    
    knn = neighbors.KNeighborsClassifier(n_neighbors = num)
    y_pred = knn.fit(X_train, y_train).predict(X_test)
    y_port = y_test_in[y_pred == 1]
    rv = y_port.mean()
    
    return rv    

In [13]:
def get_best_neighbour(num):
    rv_max = 0
    max_id = 1
    for i in range(1, num):
        value = get_neighbour_mean(i)
        if (value != None) and (value > rv_max):
            max_id = i
            rv_max = value
        print(i, value)
    return max_id, rv_max

In [16]:
best_n = get_best_neighbour(100)

In [17]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 50)
y_pred = knn.fit(X_train, y_train).predict(X_test)

In [18]:
result_matrix = confusion_matrix(y_test, y_pred)
result_matrix

array([[6363,   38],
       [1757,   13]], dtype=int64)

In [19]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.78      0.99      0.88      6401
          1       0.25      0.01      0.01      1770

avg / total       0.67      0.78      0.69      8171



In [20]:
y_port = y_test_in[y_pred == 1]
y_port.mean()

0.11409276132549026

In [21]:
y_port.shape

(51,)

In [24]:
KNN_mod = pd.DataFrame(y_pred, columns = ['KNN'])

In [30]:
KNN_mod['KNN'].value_counts()

0    8120
1      51
Name: KNN, dtype: int64

In [31]:
KNN_mod.to_json('KNN_mod.json', orient='values')

In [34]:
var_filter = ['kospi_3_sq', 'price_1', 'morning', 'price_dif_1', \
              'mkt_change_sq', 'sell_1', 'price_volatility', 'buy_1', \
              'kospi_1', 'volume_1', 'unique_id_trend', 'variation_1', \
              'kospi_3', 'post_num_1', 'price_3', 'unique_id_1']

In [57]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 50)
y_pred = knn.fit(X_train[var_filter], y_train).predict(X_test[var_filter])

In [58]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.78      0.99      0.88      6401
          1       0.32      0.01      0.02      1770

avg / total       0.68      0.78      0.69      8171



In [59]:
y_port = y_test_in[y_pred == 1]
y_port.mean()

0.27432537406000007

In [45]:
y_port.shape

(50,)

In [46]:
KNN_mod = pd.DataFrame(y_pred, columns = ['KNN'])
KNN_mod['KNN'].value_counts()

0    8121
1      50
Name: KNN, dtype: int64

In [47]:
KNN_mod.to_json('KNN_mod.json', orient='values')